In [1]:
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support as score

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 11 columns):
Age                           466 non-null int64
Gender                        466 non-null object
Total_Bilirubin               466 non-null float64
Direct_Bilirubin              466 non-null float64
Alkaline_Phosphotase          466 non-null int64
Alamine_Aminotransferase      466 non-null int64
Aspartate_Aminotransferase    466 non-null int64
Total_Protiens                466 non-null float64
Albumin                       466 non-null float64
Albumin_and_Globulin_Ratio    463 non-null float64
Dataset                       466 non-null int64
dtypes: float64(5), int64(5), object(1)
memory usage: 40.1+ KB


In [4]:
data.head().T

,0,1,2,3,4
Age,60,31,60,34,26
Gender,Male,Female,Male,Male,Female
Total_Bilirubin,6.3,0.8,19.6,8.7,0.9
Direct_Bilirubin,3.2,0.2,9.5,4,0.2
Alkaline_Phosphotase,314,158,466,298,154
Alamine_Aminotransferase,118,21,46,58,16
Aspartate_Aminotransferase,114,16,52,138,12
Total_Protiens,6.6,6,6.1,5.8,7
Albumin,3.7,3,2,2.4,3.5
Albumin_and_Globulin_Ratio,1.27,1,0.4,0.7,1


In [5]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,466.0,45.321888,15.697936,4.0,33.0,46.00,58.000,85.0
Total_Bilirubin,466.0,3.406438,6.510448,0.4,0.8,1.00,2.575,75.0
Direct_Bilirubin,466.0,1.503004,2.823700,0.1,0.2,0.30,1.300,19.7
Alkaline_Phosphotase,466.0,294.877682,252.718859,63.0,176.0,210.00,298.000,2110.0
Alamine_Aminotransferase,466.0,82.633047,192.801900,10.0,23.0,36.00,60.000,2000.0
Aspartate_Aminotransferase,466.0,114.982833,312.869109,10.0,25.0,42.00,88.000,4929.0
Total_Protiens,466.0,6.491845,1.096455,2.7,5.8,6.60,7.200,9.6
Albumin,466.0,3.149356,0.805285,0.9,2.6,3.10,3.800,5.5
Albumin_and_Globulin_Ratio,463.0,0.954104,0.333646,0.3,0.7,0.96,1.100,2.8
Dataset,466.0,1.285408,0.452094,1.0,1.0,1.00,2.000,2.0


In [6]:
disease, no_disease = data['Dataset'].value_counts()
male, female = data['Gender'].value_counts()
info=['Diognised with Liver Disease', 'Not Diognised with Liver Disease', 'Male', 'Female']
count=[disease, no_disease, male, female]
df_patient=pd.DataFrame({'Patient Info': info, 'Count': count})

In [7]:
df_patient

,Patient Info,Count
0,Diognised with Liver Disease,333
1,Not Diognised with Liver Disease,133
2,Male,361
3,Female,105


In [8]:
data['Albumin_and_Globulin_Ratio'].fillna(data['Albumin_and_Globulin_Ratio'].median(), inplace=True)

In [9]:
target=data['Dataset']
sex=pd.get_dummies(data['Gender'])
data.insert(loc=0, column='Male', value=sex['Male'])
data.insert(loc=0, column='Female', value=sex['Female'])
data.drop(['Gender'], axis=1, inplace=True)

#data.drop(['Dataset'], axis=1, inplace=True)

In [10]:
cols = data.columns
cols = list(set(cols))

del cols[cols.index('Dataset')]

del cols[cols.index('Male')]

del cols[cols.index('Female')]


In [11]:
X = data.drop(['Dataset'], axis=1)
y = data.Dataset

cols = data.columns
cols = list(set(cols))
del cols[cols.index('Dataset')]

In [12]:
data2 = pd.read_csv('test.csv')
cols2 = data2.columns
cols2 = list(set(cols2))

In [13]:
data2['Albumin_and_Globulin_Ratio'].fillna(data2['Albumin_and_Globulin_Ratio'].median(), inplace=True)

gender=pd.get_dummies(data2['Gender'])
data2.insert(loc=0, column='Male', value=gender['Male'])

data2.insert(loc=0, column='Female', value=gender['Female'])

data2.drop(['Gender'], axis=1, inplace=True)

data2.insert(loc=10, column='Dataset',value=0)
X2 = data2.drop(['Dataset'], axis=1)
y2 = data2.Dataset


In [14]:
def LR(X_train, y_train, X_test, y_test):
    
    clf = LogisticRegression()
    grid_values = {'penalty': ['l1','l2'], 'C': [0.001,0.01,0.1,1,10,100,1000]}
    
    lr = GridSearchCV(clf, param_grid=grid_values, cv=3, n_jobs=-1, scoring="f1")
    lr.fit(X_train, y_train)

    # make predictions on test data
    y_pred = lr.predict(X_test)

    print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, y_test)))
    print ("Classification report:\n{}".format(classification_report(y_test,y_pred)))
    
    precision,recall,fscore,support=score(y_test, y_pred)
    
    
    X_test2 = data2[cols]
    y_pred2 = lr.predict(X_test2)
    
    
    return fscore, accuracy_score(y_test, y_pred),y_pred2


In [15]:
def KNN(X_train, y_train, X_test, y_test):
    
    reg=KNeighborsClassifier(n_neighbors=8)

    #lr = GridSearchCV(clf, param_grid=grid_values, scoring="f1")

    k_range = list(range(1, 31))
    param_grid = dict(n_neighbors=k_range)

    grid = GridSearchCV(reg, param_grid, cv=3, n_jobs=-1, scoring='f1')
    grid.fit(X_train, y_train)
       
    print('Accuracy of KNeighbors classifier on test set: {:.2f}'.format(grid.score(X_test, y_test)))
    print ("Classification report:\n{}".format(classification_report(y_test,grid.predict(X_test))))
    
    precision,recall,fscore,support=score(y_test, grid.predict(X_test))
    return fscore, accuracy_score(y_test, grid.predict(X_test))

In [16]:
model = ["KN Neighbors", "Logistic Regression"]
fsc1=[]
fsc2=[]
acc=[]
f1acc=[]
result = ["Regular", "MinMaxScaled", "Quantile", "Max"]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X[cols], y, test_size = 0.3, random_state=42, stratify=y)


fscore, accuracy = KNN(X_train, y_train, X_test, y_test)

fsc1.append(fscore[0])
fsc2.append(fscore[1])
acc.append(accuracy)

fscore, accuracy,dat= LR(X_train, y_train, X_test, y_test)

fsc1.append(fscore[0])
fsc2.append(fscore[1])
acc.append(accuracy)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Accuracy of KNeighbors classifier on test set: 0.78
Classification report:
              precision    recall  f1-score   support

           1       0.73      0.83      0.78       100
           2       0.37      0.25      0.30        40

   micro avg       0.66      0.66      0.66       140
   macro avg       0.55      0.54      0.54       140
weighted avg       0.63      0.66      0.64       140

Accuracy of logistic regression classifier on test set: 0.83
Classification report:
              precision    recall  f1-score   support

           1       0.71      1.00      0.83       100
           2       0.00      0.00      0.00        40

   micro avg       0.71      0.71      0.71       140
   macro avg       0.36      0.50      0.42       140
weighted avg       0.51      0.71      0.60       140



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\s

In [18]:
scaler=MinMaxScaler()

X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])
X_test[X_test.columns] = scaler.transform(X_test[X_test.columns])

fscore, accuracy = KNN(X_train, y_train, X_test, y_test)

fsc1.append(fscore[0])
fsc2.append(fscore[1])
acc.append(accuracy)

fscore, accuracy,dat = LR(X_train, y_train, X_test, y_test)

fsc1.append(fscore[0])
fsc2.append(fscore[1])
acc.append(accuracy)

dat=pd.DataFrame(dat)
dat.to_csv('test.csv')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


Accuracy of KNeighbors classifier on test set: 0.79
Classification report:
              precision    recall  f1-score   support

           1       0.72      0.88      0.79       100
           2       0.33      0.15      0.21        40

   micro avg       0.67      0.67      0.67       140
   macro avg       0.53      0.52      0.50       140
weighted avg       0.61      0.67      0.63       140

Accuracy of logistic regression classifier on test set: 0.83
Classification report:
              precision    recall  f1-score   support

           1       0.71      1.00      0.83       100
           2       0.00      0.00      0.00        40

   micro avg       0.71      0.71      0.71       140
   macro avg       0.36      0.50      0.42       140
weighted avg       0.51      0.71      0.60       140



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\s